In [151]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BartForConditionalGeneration, BartTokenizer

In [152]:
MODEL_NAME = "sbtraining2020/email_bart_1"

In [153]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [155]:
from transformers import AutoTokenizer, AutoModelForCausalLM
hf_tokenizer =BartTokenizer.from_pretrained(MODEL_NAME) # AutoTokenizer.from_pretrained( MODEL_NAME) # "sbtraining2020/esubjectgen_llama31_clean")
hf_model = BartForConditionalGeneration.from_pretrained(MODEL_NAME) #"sbtraining2020/esubjectgen_llama31_clean")

In [167]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hf_model.to(device)
#hf_model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [168]:
def get_email_subject(text, max_length=500):
    """
    Generates a email subject for the given text using a pre-trained model.

    Args:
        text (str): Email text to generate subject.
        max_length (int): The maximum length of the input text for the model.

    Returns:
        str: The generated subject for the input text.
    """
    # Encode the input text using the tokenizer. The 'pt' indicates PyTorch tensors.
    inputs = hf_tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=False)

    # Move the encoded text to the same device as the model (e.g., GPU or CPU)
    inputs = inputs.to(device)

    # Generate summary IDs with the model. num_beams controls the beam search width.
    # early_stopping is set to False for a thorough search, though it can be set to True for faster results.
    subject_ids = hf_model.generate(inputs, max_length=500, num_beams=30, early_stopping=False)

    # Decode the generated IDs back to text, skipping special tokens like padding or EOS.
    subject = hf_tokenizer.decode(subject_ids[0], skip_special_tokens=True)

    # Return the generated subject
    return subject

In [169]:
email_text = """Plove is going to go to Dallas.
We are going to leave next Friday when he  gets done (7ish) and go up for the game.
The game is at 11 in the morning,  so we will come home directly after it.
Plove says he has a friend who has a  place in Dallas that we can crash at if we don't want to pay for a hotel.
Do you want to go?
        """

In [170]:
email_text = """
Michelle Here are my very minor comments.
However we still need to wait on any additions, based on meeting with SME's today.
One concern is the firing of the learner who performs  bad in the final two scenarios.
Do we face any copyright issues using the CNN type themes?
In addition, I think we need to stay clear of anything that remotely seems like California or anything that really happen with Enron?
(i.e.So-cal Waha) In addition, comments on regulatory issues may be a problem (i.e.California Legislature).
Sheri  When you read all the scripts together and due to the similar mechanics being taught it appears very repetitious.
Thus I do believe we need to maybe use a "Dateline" type theme for one, and a "60 Minute" type theme for another scenario vice just the CNN type theme.
In the last two scenarios can we include a promotion out of the associate program for the stellar performers (i.e.title change to manager)?
Cheers Kirk
"""

In [171]:
#email_text = """The following reports have been waiting for your approval for more than 4 days. Please review. Owner: James W Reitmeyer Report Name: JReitmeyer 10/24/01 Days In Mgr. Queue: 5 """


In [172]:
result = get_email_subject(email_text)

1

In [166]:
result

"Comments from SME's"